```{contents}
```
## Tool Registry 

The **Tool Registry** in LangGraph is the centralized, structured mechanism for **defining, storing, validating, selecting, and invoking external tools** inside graph-based LLM workflows.
It acts as the **interface layer between reasoning (LLMs) and action (external systems)**.

---

### **1. Why Tool Registry Exists**

LLMs cannot directly perform real-world actions (API calls, database queries, computations).
They must invoke **tools** safely and deterministically.

| Problem       | Without Registry   | With Registry        |
| ------------- | ------------------ | -------------------- |
| Tool chaos    | Ad-hoc functions   | Controlled catalog   |
| Security risk | Unrestricted calls | Explicit permissions |
| Debugging     | Hard               | Fully traceable      |
| Scalability   | Fragile            | Modular & extensible |

---

### **2. Conceptual Architecture**

```
LLM → Tool Selection → Tool Registry → Tool Execution → State Update
```

The **Tool Registry** enforces:

* Which tools exist
* What parameters they accept
* How results update state
* Who is allowed to call them

---

### **3. Core Components of a Tool Registry**

| Component        | Role                      |
| ---------------- | ------------------------- |
| Tool Definition  | Name, description, schema |
| Parameter Schema | Typed inputs              |
| Validator        | Enforces correctness      |
| Executor         | Runs the tool             |
| Result Adapter   | Converts output to state  |
| Access Policy    | Controls who can call     |

---

### **4. Tool Definition Structure**

```python
from langchain.tools import tool

@tool
def search(query: str) -> str:
    """Search the web for relevant information."""
    return f"Results for {query}"
```

Each tool has:

* **Name**
* **Docstring** (used by LLM for reasoning)
* **Typed signature**
* **Deterministic behavior**

---

### **5. Registering Tools in LangGraph**

```python
from langgraph.prebuilt import ToolNode

tools = [search]
tool_node = ToolNode(tools)
```

This creates a **tool registry node** in the graph.

---

### **6. Tool Invocation Workflow**

```
LLM Node
   ↓ (tool_call)
Tool Registry Node
   ↓ (validate & execute)
State Update
   ↓
Next Node
```

Execution is **deterministic** and **fully logged**.

---

### **7. Example Graph with Tool Registry**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI

class State(TypedDict):
    messages: list

llm = ChatOpenAI()
tools = [search]

tool_node = ToolNode(tools)

def llm_node(state):
    response = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}

builder = StateGraph(State)
builder.add_node("llm", llm_node)
builder.add_node("tools", tool_node)

builder.add_edge("llm", "tools")
builder.add_edge("tools", "llm")

builder.set_entry_point("llm")
graph = builder.compile()
```

This creates a **reason–act loop**.

---

### **8. Tool Selection Logic**

The LLM does not execute tools directly.
It outputs a **tool call request**:

```json
{
  "name": "search",
  "arguments": { "query": "LangGraph tool registry" }
}
```

The **Tool Registry**:

1. Validates schema
2. Executes tool
3. Captures output
4. Injects result into state

---

### **9. Advanced Registry Features**

| Feature           | Purpose                  |
| ----------------- | ------------------------ |
| Schema validation | Prevent malformed calls  |
| Tool permissions  | Restrict sensitive tools |
| Sandboxing        | Secure execution         |
| Rate limiting     | Prevent abuse            |
| Timeout control   | Avoid hangs              |
| Retry policies    | Fault tolerance          |
| Audit logging     | Compliance               |
| Cost tracking     | Budget control           |

---

### **10. Variants of Tool Registry Usage**

| Variant                | Use Case                     |
| ---------------------- | ---------------------------- |
| Static registry        | Fixed tool set               |
| Dynamic registry       | Tools added at runtime       |
| Per-agent registry     | Each agent has its own tools |
| Hierarchical registry  | Tools grouped by domain      |
| Policy-driven registry | Role-based access            |

---

### **11. Why Tool Registry is Critical**

Without a tool registry, LLM systems are:

* unsafe
* non-deterministic
* unscalable
* ungovernable

With it, they become **production-grade AI systems**.

---

### **12. Mental Model**

Think of the Tool Registry as:

> **Operating system kernel for AI actions**

It controls all interactions between the model and the real world.

---

If you'd like next, I can explain:

• Tool routing and multi-tool orchestration
• Security & sandboxing strategies
• How to design a production-grade tool registry



### Demonstration

In [5]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# Reducer
def add_messages(a: List[BaseMessage], b: List[BaseMessage]):
    return a + b

class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# Tool
@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression."""
    return str(eval(expression))

tool_node = ToolNode([calculator])
llm = ChatOpenAI()

def llm_node(state: State):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

# Routing function (NOT a node)
def route(state: State):
    last = state["messages"][-1].content.lower()
    if "final answer" in last:
        return END
    if "calculator" in last:
        return "tools"
    return "llm"

builder = StateGraph(State)

builder.add_node("llm", llm_node)
builder.add_node("tools", tool_node)

builder.set_entry_point("llm")

builder.add_conditional_edges(
    "llm",
    route,
    {"tools": "tools", "llm": "llm", END: END}
)

builder.add_edge("tools", "llm")

graph = builder.compile()

# Run
result = graph.invoke(
    {"messages": [HumanMessage(content="Calculate 25 * 48 using the calculator and give Final Answer.")]},
    config={"recursion_limit": 10}
)

print(result["messages"][-1].content)


25 * 48 = 1200

Final Answer: 1200
